# 🧪 Sentiment Analysis mit RoBERTa + MongoDB (.env Unterstützung)
Dieses Notebook liest Tweets aus einer MongoDB-Datenbank, analysiert sie mit einem RoBERTa-Modell und kann die Ergebnisse in der Datenbank speichern.

## 1. 📦 Setup & Imports

In [1]:
import os
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

/Users/yanickfischer/Documents/vsCode/MDM/project1/ukraineBias/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 🔐 Verbindung zur MongoDB mit `.env`

In [2]:
def connect_to_mongo():
    load_dotenv()
    uri = os.getenv("MONGO_URI")
    if not uri:
        raise ValueError("MONGO_URI fehlt!")
    try:
        client = MongoClient(uri, serverSelectionTimeoutMS=3000)
        client.admin.command("ping")
        print("✅ Verbindung zu MongoDB erfolgreich")
        db = client["ukraineBiasDB"]
        collection = db["tweets_balanced"]
        return collection
    except ConnectionFailure as e:
        print(f"❌ Fehler bei Verbindung: {e}")
        return None

# Verbindung testen
collection = connect_to_mongo()

✅ Verbindung zu MongoDB erfolgreich


## 3. 🤗 RoBERTa Modell laden

In [3]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = ['negative', 'neutral', 'positive']

## 4. 🔍 Inferenzfunktion definieren

In [4]:
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = torch.argmax(logits).item()
    return labels[predicted_class_id]

## 5. 🧪 Analyse von MongoDB-Tweets (Vorschau)

In [5]:
if collection:
    for doc in collection.find().limit(5):
        text = doc.get("text", "")
        sentiment = get_sentiment(text)
        print(f"Text: {text}\n→ Sentiment: {sentiment}\n")

NotImplementedError: Collection objects do not implement truth value testing or bool(). Please compare with None instead: collection is not None

## 6. 💾 Optional: Sentiment zurück in MongoDB schreiben

In [ ]:
# Achtung: Ändert deine Datenbank
if collection:
    for doc in collection.find():
        text = doc.get("text", "")
        sentiment = get_sentiment(text)
        collection.update_one({"_id": doc["_id"]}, {"$set": {"sentiment": sentiment}})